In [1]:
import os
import sys
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import polars as pl
from omegaconf import OmegaConf
from PIL import Image
from tqdm.auto import tqdm
from transformers import pipeline

from src.config import cfg
from src.dir import create_dir
from src.seed import seed_everything

cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)
pl.Config.set_fmt_str_lengths(1000)


/home/marumarukun/pj/compe/atma_18/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


exp_number: '007'
run_time: base
data:
  input_root: ../../data/input
  train_path: ../../data/input/train_features.csv
  test_path: ../../data/input/test_features.csv
  sample_submission_path: ../../data/input/sample_submission.csv
  img_root: ../../data/input/images
  output_root: ../../data/output
  results_root: ../../results
  results_path: ../../results/007/base
seed: 319
n_splits: 4
target_cols:
- x_0
- y_0
- z_0
- x_1
- y_1
- z_1
- x_2
- y_2
- z_2
- x_3
- y_3
- z_3
- x_4
- y_4
- z_4
- x_5
- y_5
- z_5
cnn:
  model_name: tf_efficientnet_b0_ns
  size: 224
  pretrained: true
  in_chans: 9
  target_size: 18
  lr: 0.001
  num_epochs: 20
  batch_size: 32



polars.config.Config

In [2]:
# データの読み込み
train = pl.read_csv(cfg.data.train_path, try_parse_dates=True)
test = pl.read_csv(cfg.data.test_path, try_parse_dates=True)
sample_submission = pl.read_csv(cfg.data.sample_submission_path, try_parse_dates=True)

# データの結合(label encoding用)
train_test = pl.concat([train, test], how="diagonal")

# scene列を作成 → これでGroupKFoldする
train = train.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))
test = test.with_columns(pl.col("ID").str.split("_").list[0].alias("scene"))


In [3]:
def get_relative_path(path):
    return os.path.join(cfg.data.img_root, path)

# 画像へのパス
image_path_root_list = [
    get_relative_path("{ID}/image_t.png"),
    get_relative_path("{ID}/image_t-0.5.png"),
    get_relative_path("{ID}/image_t-1.0.png"),
]

# Depth Mapの生成と保存
depth_anything_v2 = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf", device="cuda")

for row in tqdm(train_test.iter_rows(named=True)):
    for image_path_root in image_path_root_list:
        img_pil = Image.open(image_path_root.format(ID=row["ID"]))
        pred = depth_anything_v2(img_pil)

        depth_path = image_path_root.format(ID=row["ID"]).replace("images", "depth")
        os.makedirs(os.path.dirname(depth_path), exist_ok=True)
        pred["depth"].save(depth_path)


1it [00:00,  3.00it/s]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
45098it [33:56, 22.15it/s]
